In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
import dataset

In [ ]:
root_dir = '/Volumes/Transcend/Data/Steel/'
csv_file = root_dir + 'train.csv'
train_dir = root_dir + 'train/'
mask_dir = root_dir + 'mask/'

In [ ]:
# img_dataset = dataset.gen_dataset()

In [ ]:
img_dataset = tf.data.Dataset.from_generator(dataset.gen_dataset, (tf.float32, tf.float32), (tf.TensorShape([256, 1600, 3]), tf.TensorShape([256, 1600, 4])))

In [ ]:
img_dataset = img_dataset.take(4)
img_dataset = img_dataset.batch(2)

In [ ]:
import importlib

In [ ]:
import BaseUNet

In [ ]:
importlib.reload(BaseUNet)

In [ ]:
model = BaseUNet.UNet()

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_iou = tf.keras.metrics.MeanIoU(num_classes=4)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./log_dir/', histogram_freq=1)

In [ ]:
def dice(y_true, y_pred):
    smooth = 1
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true * y_pred)
    return (2 * intersection + smooth) / (tf.keras.backend.sum(y_true) + tf.keras.backend.sum(y_pred) + smooth)

In [ ]:
model.compile(optimizer=optimizer,
              loss = loss_obj,
              metrics=[dice])

In [ ]:
model.fit(img_dataset, steps_per_epoch=2, epochs=2, callbacks=[callback, tensorboard_callback])

In [ ]:
def get_iou_vector(A, B):
    # Numpy version
    A = np.array(A)
    B = np.array(B)
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def my_iou_metric(label, pred):
    # Tensorflow version
    return tf.py_function(get_iou_vector, [label, pred > 0.5], tf.float64)

In [ ]:
iou = []
@tf.function
def train(dataset, loss_obj, train_loss, train_metric, model, optimizer):
    count = 0
    for image, masks in img_dataset:
        with tf.GradientTape() as tape:
            predictions = model(image)
            loss = loss_obj(masks, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        dice(masks, predictions)
        my_iou_metric(masks, predictions)

In [ ]:
train(img_dataset, loss_obj, train_loss, train_iou, model, optimizer)

In [ ]:
assert False

In [ ]:
img_dataset = img_dataset.take(10)
img_dataset = img_dataset.repeat()
img_dataset = img_dataset.batch(2)

In [ ]:
count = 0
for image, label_image in img_dataset:
    out = model(image)
    assert False

In [ ]:
a = np.zeros((5, 3, 3, 4))

In [ ]:
b = np.ones((5, 3, 3, 4))

In [ ]:
ta = tf.convert_to_tensor(a)
tb = tf.convert_to_tensor(b)

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=4)

In [ ]:
m.update_state(a, b)

In [ ]:
out_r = tf.round(tf.constant(out))

In [ ]:
out_r.shape

In [ ]:
tm = tf.convert_to_tensor(np.ones((2, 256, 1600, 4)))

In [ ]:
tm.shape

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=4)

In [ ]:
m.update_state(out_r, tm)

In [ ]:
out_r = tf.reshape(out_r, [-1])

In [ ]:
out_r.shape

In [ ]:
label_image.shape

In [ ]:
label_image = tf.reshape(label_image, [-1, 4])

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=4)

In [ ]:
m.update_state(label_image, out_r)

In [ ]:
m.result()

In [ ]:
tf.argmax(out, axis=).shape

In [ ]:
mask = tf.constant(label_image)

In [ ]:
mask.shape

In [ ]:
count

In [ ]:
label_image.shape.ndims

In [ ]:
r_label = tf.reshape(label_image, [-1])

In [ ]:
r_label.shape